In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install transformers datasets transformers[torch] accelerate>=0.20.1

In [101]:
import os
import time
from tqdm.notebook import tqdm
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F

path_to_model = '/content/drive/MyDrive/model_compression/my_model'
processor = AutoFeatureExtractor.from_pretrained(path_to_model)
vit_model = AutoModelForImageClassification.from_pretrained(path_to_model)

def model_use(model, img):
    with torch.no_grad():
        logits = model(**img).logits
    pred_label = logits.argmax(-1).item()
    return model.config.id2label[pred_label]

images_list = os.listdir('/content/drive/MyDrive/model_compression/data')

start = time.time()
target_lst = []
predict_lst = []
logits_lst = []

for img_name in images_list:
    img_path = os.path.join('/content/drive/MyDrive/model_compression/data', img_name)
    image = Image.open(img_path, mode='r')
    inputs = processor(image, return_tensors="pt")
    predicts, logits = model_use(vit_model, inputs)
    target = img_name[:img_name.find(".")]
    if target == "dog":
        label = 1
    else:
        label = 0
    target_lst.append(label)
    if predicts == "dog":
        pr = 1
    else:
        pr = 0
    predict_lst.append(pr)
    logits_lst.append(logits)

end = time.time()
acc = accuracy_score(target_lst, predict_lst)

print("accuracy исходной модели= ", acc)
print("Время обработки изображений исходной модели= ", end-start, " секунд")
print("Скорость обработки изображений у исходной модели составила  ", len(images_list)/(end-start), " картинок в секунду")
infer_time = ((end - start) / len(images_list)) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


accuracy исходной модели=  0.9625
Время обработки изображений исходной модели=  194.9379551410675  секунд
Скорость обработки изображений у исходной модели составила   0.8207739733609879  картинок в секунду
Avg inference time: 1218.3622 ms


In [114]:
'''
Cчитаем soft labels с температурой T.
Более высокая температура приводит к более "мягким" меткам,
которые имеют большую разницу между вероятностями классов.

T = 2.0 - Умеренно "мягкие" метки.

'''
T = 2.0
soft_labels = F.softmax(torch.cat(logits_lst, dim=0) / T, dim=1)

# Наша слабая модель (пример с ResNet-18)
student_model = models.resnet18(pretrained=False)
student_model.fc = nn.Linear(student_model.fc.in_features, 2)  # 2 класса: cat и dog

# Loss с учетом Knowledge Distillation
def distillation_loss(outputs_student, outputs_teacher, alpha=0.5, temperature=1.0):
    hard_loss = F.cross_entropy(outputs_student, outputs_teacher.argmax(-1))
    soft_loss = nn.KLDivLoss()(F.log_softmax(outputs_student / temperature, dim=1),
                               F.softmax(outputs_teacher / temperature, dim=1))
    return (1 - alpha) * hard_loss + alpha * temperature**2 * soft_loss

# еще и шедулер добавим
optimizer = optim.Adam(student_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Учим студента
num_epochs = 20

student_model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for img_name, soft_label in zip(images_list, soft_labels):
        img_path = os.path.join('/content/drive/MyDrive/model_compression/data', img_name)
        image = Image.open(img_path, mode='r')

        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = transform(image)

        optimizer.zero_grad()

        outputs_student = student_model(image.unsqueeze(0)) # добавим измерение батча
        outputs_teacher = soft_label.unsqueeze(0)  # тензор с размером батча 1

        loss = distillation_loss(outputs_student, outputs_teacher)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss}')




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 1/20, Loss: 1.422323614358902
Epoch 2/20, Loss: 0.3584473133087158
Epoch 3/20, Loss: 0.37765002250671387
Epoch 4/20, Loss: 0.2627972811460495
Epoch 5/20, Loss: 0.2615549564361572
Epoch 6/20, Loss: 0.25926532596349716
Epoch 7/20, Loss: 0.2586737275123596
Epoch 8/20, Loss: 0.25841058790683746
Epoch 9/20, Loss: 0.25850844383239746
Epoch 10/20, Loss: 0.258448526263237
Epoch 11/20, Loss: 0.2582792341709137
Epoch 12/20, Loss: 0.25835975259542465
Epoch 13/20, Loss: 0.2582198232412338
Epoch 14/20, Loss: 0.25832997262477875
Epoch 15/20, Loss: 0.25821061432361603
Epoch 16/20, Loss: 0.25827697664499283
Epoch 17/20, Loss: 0.25821562111377716
Epoch 18/20, Loss: 0.25824691355228424
Epoch 19/20, Loss: 0.2582192122936249
Epoch 20/20, Loss: 0.25822728127241135


In [115]:
# Оценка студента
student_model.eval()
student_preds = []

start = time.time()

for img_name in images_list:
    img_path = os.path.join('/content/drive/MyDrive/model_compression/data', img_name)
    image = Image.open(img_path, mode='r')

    image = transform(image)

    with torch.no_grad():
        logits = student_model(image.unsqueeze(0))
    pred_label = logits.argmax(-1).item()
    student_preds.append(pred_label)

end = time.time()

student_acc = accuracy_score(target_lst, student_preds)

print("Accuracy студента= ", student_acc)
print("Время обработки изображений студентской моделью= ", end-start, " секунд")
print("Скорость обработки изображений у студентской модели составила  ", len(images_list)/(end-start), " картинок в секунду")

infer_time = ((end - start) / len(images_list)) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')


Accuracy студента=  0.5
Время обработки изображений студентской моделью=  30.631128072738647  секунд
Скорость обработки изображений у студентской модели составила   5.223444582911008  картинок в секунду
Avg inference time: 191.4446 ms
